In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import torch.nn as nn
from torchvision.datasets import CIFAR10
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torch.nn.functional as F

In [3]:
data = np.load('CWRU_dataset.npy')
labels = np.load('CWRU_lables.npy')

In [4]:
x=data[:,0:1600]

In [5]:
def mean(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.mean(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def median(data,no_elements):
    X=np.zeros((data.shape[0],data.shape[1]))
    for i in range(data.shape[1]-no_elements+1):
        X[:,i]=np.median(data[:,i:i+no_elements],axis=1)
    return X.astype(np.float16)
def sig_image(data,size):
    X=np.zeros((data.shape[0],size,size))
    for i in range(data.shape[0]):
        X[i]=(data[i,:].reshape(size,size))
    return X.astype(np.float16)

In [6]:
channel_mean=(mean(x,10)).astype(np.float16)
x_m=sig_image(channel_mean,40)
channel_median=(median(x,10)).astype(np.float16)
x_md=sig_image(x,40)

In [7]:
x_n=sig_image(x,40)

In [8]:
x_n.shape

(2800, 40, 40)

In [9]:
x_m.shape

(2800, 40, 40)

In [10]:
X=np.stack((x_n,x_m,x_md),axis=1).astype(np.float16)

In [11]:
X.shape

(2800, 3, 40, 40)

In [12]:
from sklearn.model_selection import train_test_split
trainx, testx, trainlabel, testlabel = train_test_split(X, labels, test_size=0.2, random_state=20)

In [13]:
sig_train, sig_test = trainx,testx
lab_train, lab_test = trainlabel,testlabel

In [14]:
sig_train = torch.from_numpy(sig_train)
sig_test = torch.from_numpy(sig_test)
lab_train= torch.from_numpy(lab_train)
lab_test = torch.from_numpy(lab_test)

In [15]:
import torch.utils.data as data_utils
batch_size = 128 
train_tensor = data_utils.TensorDataset(sig_train, lab_train) 
train_loader = data_utils.DataLoader(dataset = train_tensor, batch_size = batch_size, shuffle = True)

In [16]:
batch_size = 1024
test_tensor = data_utils.TensorDataset(sig_test, lab_test) 
test_loader = data_utils.DataLoader(dataset = test_tensor, batch_size = batch_size, shuffle = False)

In [17]:
sig_train.size()

torch.Size([2240, 3, 40, 40])

In [18]:
sig_test.size()

torch.Size([560, 3, 40, 40])

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=4,stride=1,padding = 1)
        self.mp1 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.conv2 = nn.Conv2d(32,64, kernel_size=4,stride =1)
        self.mp2 = nn.MaxPool2d(kernel_size=4,stride=2)
        self.fc1= nn.Linear(2304,256)
        self.dp1 = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256,10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp1(self.conv1(x)))
        x = F.relu(self.mp2(self.conv2(x)))
        x = x.view(in_size,-1)
        x = F.relu(self.fc1(x))
        x = self.dp1(x)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [21]:
cnn = CNN().double()

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)

In [23]:
num_epochs = 100

In [24]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (signals, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        
        loss.backward()
        optimizer.step()
        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list.append(correct / total)

        if (epoch+1) % 5 == 0 or epoch==0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))
        

Epoch [1/100], Step [1/18], Loss: 2.3026, Train Accuracy: 15.62%
Epoch [1/100], Step [2/18], Loss: 2.1436, Train Accuracy: 12.50%
Epoch [1/100], Step [3/18], Loss: 2.2310, Train Accuracy: 7.81%
Epoch [1/100], Step [4/18], Loss: 2.1341, Train Accuracy: 17.19%
Epoch [1/100], Step [5/18], Loss: 1.9999, Train Accuracy: 17.97%
Epoch [1/100], Step [6/18], Loss: 2.1688, Train Accuracy: 12.50%
Epoch [1/100], Step [7/18], Loss: 1.9400, Train Accuracy: 21.09%
Epoch [1/100], Step [8/18], Loss: 1.9452, Train Accuracy: 21.88%
Epoch [1/100], Step [9/18], Loss: 1.8811, Train Accuracy: 20.31%
Epoch [1/100], Step [10/18], Loss: 1.8257, Train Accuracy: 31.25%
Epoch [1/100], Step [11/18], Loss: 1.8228, Train Accuracy: 29.69%
Epoch [1/100], Step [12/18], Loss: 1.8597, Train Accuracy: 22.66%
Epoch [1/100], Step [13/18], Loss: 1.8749, Train Accuracy: 20.31%
Epoch [1/100], Step [14/18], Loss: 1.6941, Train Accuracy: 30.47%
Epoch [1/100], Step [15/18], Loss: 1.7534, Train Accuracy: 34.38%
Epoch [1/100], Step 

Epoch [30/100], Step [17/18], Loss: 0.1192, Train Accuracy: 95.31%
Epoch [30/100], Step [18/18], Loss: 0.0472, Train Accuracy: 96.88%
Epoch [35/100], Step [1/18], Loss: 0.0985, Train Accuracy: 96.88%
Epoch [35/100], Step [2/18], Loss: 0.0181, Train Accuracy: 100.00%
Epoch [35/100], Step [3/18], Loss: 0.0432, Train Accuracy: 99.22%
Epoch [35/100], Step [4/18], Loss: 0.0185, Train Accuracy: 100.00%
Epoch [35/100], Step [5/18], Loss: 0.0700, Train Accuracy: 95.31%
Epoch [35/100], Step [6/18], Loss: 0.0484, Train Accuracy: 98.44%
Epoch [35/100], Step [7/18], Loss: 0.0607, Train Accuracy: 97.66%
Epoch [35/100], Step [8/18], Loss: 0.0152, Train Accuracy: 99.22%
Epoch [35/100], Step [9/18], Loss: 0.1009, Train Accuracy: 96.09%
Epoch [35/100], Step [10/18], Loss: 0.0694, Train Accuracy: 96.09%
Epoch [35/100], Step [11/18], Loss: 0.0466, Train Accuracy: 97.66%
Epoch [35/100], Step [12/18], Loss: 0.1283, Train Accuracy: 93.75%
Epoch [35/100], Step [13/18], Loss: 0.0505, Train Accuracy: 97.66%
Ep

Epoch [65/100], Step [15/18], Loss: 0.0054, Train Accuracy: 100.00%
Epoch [65/100], Step [16/18], Loss: 0.0012, Train Accuracy: 100.00%
Epoch [65/100], Step [17/18], Loss: 0.0084, Train Accuracy: 100.00%
Epoch [65/100], Step [18/18], Loss: 0.0035, Train Accuracy: 100.00%
Epoch [70/100], Step [1/18], Loss: 0.0019, Train Accuracy: 100.00%
Epoch [70/100], Step [2/18], Loss: 0.0048, Train Accuracy: 100.00%
Epoch [70/100], Step [3/18], Loss: 0.0028, Train Accuracy: 100.00%
Epoch [70/100], Step [4/18], Loss: 0.0013, Train Accuracy: 100.00%
Epoch [70/100], Step [5/18], Loss: 0.0029, Train Accuracy: 100.00%
Epoch [70/100], Step [6/18], Loss: 0.0018, Train Accuracy: 100.00%
Epoch [70/100], Step [7/18], Loss: 0.0028, Train Accuracy: 100.00%
Epoch [70/100], Step [8/18], Loss: 0.0037, Train Accuracy: 100.00%
Epoch [70/100], Step [9/18], Loss: 0.0015, Train Accuracy: 100.00%
Epoch [70/100], Step [10/18], Loss: 0.0057, Train Accuracy: 100.00%
Epoch [70/100], Step [11/18], Loss: 0.0015, Train Accurac

Epoch [100/100], Step [11/18], Loss: 0.0012, Train Accuracy: 100.00%
Epoch [100/100], Step [12/18], Loss: 0.0010, Train Accuracy: 100.00%
Epoch [100/100], Step [13/18], Loss: 0.0007, Train Accuracy: 100.00%
Epoch [100/100], Step [14/18], Loss: 0.0012, Train Accuracy: 100.00%
Epoch [100/100], Step [15/18], Loss: 0.0019, Train Accuracy: 100.00%
Epoch [100/100], Step [16/18], Loss: 0.0029, Train Accuracy: 100.00%
Epoch [100/100], Step [17/18], Loss: 0.0005, Train Accuracy: 100.00%
Epoch [100/100], Step [18/18], Loss: 0.0006, Train Accuracy: 100.00%


In [25]:
total_step = len(test_loader)
print(total_step)
loss_list_test = []
acc_list_test = []
with torch.no_grad():
    for i, (signals, labels) in enumerate(test_loader):
        # Run the forward pass
        signals=signals
        labels=labels
        outputs = cnn(signals.double())
        loss = criterion(outputs, labels.long())
        loss_list_test.append(loss.item())
        if epoch%10 ==0:
            print(loss)
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels.long()).sum().item()
        acc_list_test.append(correct / total)
        if (epoch) % 1 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

1
Epoch [100/100], Step [1/1], Loss: 0.0686, Accuracy: 98.57%


In [26]:
# if you need to save
torch.save(cnn,'cnnTC3_fold3_45.pth')

/home/cmu/anaconda3/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type CNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
